#                                            DIRECTIONAL DRILLING 
##                                           *DIRECTIONAL WELLS PROFILES*

***

<img src="resources/Well_prof.jpg" width=800 height=800 />

# Python Libraries

In [8]:
from collections import namedtuple
from math import radians, acos, asin, cos, sin, atan, degrees, sqrt

# *Directional Wells Profiles*

## *Slant Well Profile (J Type)*

<img src="resources/j_prof.png" width=500 height=500 align='center' />

In [9]:
Data = namedtuple("Input", "TVD KOP BUR DH")
Output = namedtuple("Output", "R Theta TVD_EOB Md_EOB Dh_EOB Tan_len Md_total")

def well_J(data:Data, unit='ingles') -> Output:
    # 1. Call input values
    tvd = data.TVD
    kop = data.KOP
    bur = data.BUR
    dh = data.DH
    
    # 3. Calculo de R
    if unit == 'ingles':
        R = 5729.58 / bur
    else:
        R = 1718.87 / bur
        
    # 4. Calculo de theta
    # 4.1 Calculo de dc
    if dh > R:
        dc = dh - R
    elif dh < R:
        dc = R - dh
        
    # 4.2 Calculo de DO    
    do = tvd - kop
    
    # 4.3 Angulo DOC
    doc = degrees(atan(dc / do))
    
    # 4.4 Calculo de OC
    oc = sqrt(dc**2 + do**2)
    
    # 4.5 Calculo de angulo BOC
    boc = degrees(acos(R / oc))
    
    # 4.6 Angulo BOD
    if R < dh:
        bod = boc - doc
    elif R > dh:
        bod = boc + doc
        
    # 4.7 Angulo Theta    
    theta = 90 - bod
    
    # 5. TVD @EOB
    tvd_eob = kop + abs(R * sin(radians(theta)))
    
    # 6. MD @EOB
    if unit == 'ingles':
        md_eob = kop + (theta / bur) * 100
    else:
        md_eob = kop + (theta / bur) * 30
        
    # 7. DH @EOB    
    dh_eob = R - R * cos(radians(theta))
    
    # 8. MD Total
    # 8.1 BC
    tan_len = sqrt(oc**2 - R**2)
    
    # 8.2 MD Total
    if unit == 'ingles':
        md_total = kop + (theta / bur) * 100 + tan_len
    else:
        md_total = kop + (theta / bur) * 30 + tan_len
        
    return Output(R=R, Theta=theta, TVD_EOB=tvd_eob, Md_EOB=md_eob, Dh_EOB=dh_eob, Tan_len=tan_len, Md_total=md_total)

#     names = ['R', 'theta', 'tvd_EOB', 'MD_EOB', 'DH_EOB', 'Length_tan', 'MD_Total']
#     for param , value in zip(names, trajectory_J):
#         if param == 'theta':
#             print(f"{param} -> {value:.3f} degrees")
#         else:
#             print(f"{param} -> {value:.3f} ft")

## *Ejercicio 1*

In [10]:
# data
tvd = 8000 #ft
kop = 500 #ft
bur = 2 #o/100ft
dh = 970.8 #ft

In [11]:
trajectory_J = well_J(Data(tvd, kop, bur, dh))

In [12]:
trajectory_J

Output(R=2864.79, Theta=7.564230623470863, TVD_EOB=877.1139517978513, Md_EOB=878.2115311735431, Dh_EOB=24.929649303260703, Tan_len=7185.414140882904, Md_total=8063.625672056447)

In [14]:
names = ['R', 'theta', 'tvd_EOB', 'MD_EOB', 'DH_EOB', 'Length_tan', 'MD_Total']
for param , value in zip(names, trajectory_J):
    if param == 'theta':
        print(f"{param} -> {value:.2f} degrees")
    else:
        print(f"{param} -> {value:.2f} ft")

R -> 2864.79 ft
theta -> 7.56 degrees
tvd_EOB -> 877.11 ft
MD_EOB -> 878.21 ft
DH_EOB -> 24.93 ft
Length_tan -> 7185.41 ft
MD_Total -> 8063.63 ft


## *S-Type Well Profile*

<img src="resources/s_prof.png" width=500 height=400 />

In [18]:
# Function to calculate parameters from a S-Type well
Data_S = namedtuple("Input", "TVD KOP BUR DOR DH")
Output_S = namedtuple("Output", "R1 R2 Theta TVD_EOB Md_EOB Dh_EOB Tan_len Md_SOD TVD_SOD Dh_SOD Md_total")

def well_S(data:Data_S, unit='ingles'):
    # 1. Definir variables
    tvd = data.TVD
    kop = data.KOP
    bur = data.BUR
    dor = data.DOR
    dh = data.DH
    
    # 3. R1 y R2
    if unit == 'ingles':
        R1 = 5729.58 / bur
        R2 = 5729.58 / dor
    else:
        R1 = 1718.87 / bur
        R2 = 1718.87 / dor
        
    # 4. Calcular Theta
    # 4.1. Calcular FE
    
    if dh > (R1 + R2):
        fe = dh - (R1 + R2)
    elif dh < (R1 + R2):
        fe = R1 - (dh - R2)
        
    # 4.2. Calcular EO
    eo = tvd - kop
    
    # 4.3 Calcular FOE
    foe = degrees(atan(fe / eo))
    
    # 4.4. Calcular OF
    of = sqrt(fe**2 + eo**2)
    
    # 4.5. Calcular FG
    fg = R1 + R2
    
    # 4.6. Calcular angulo FOG
    fog = degrees(asin(fg / of))
    
    # 4.7. Angulo Theta
    theta = fog - foe
    
    # 5. TVD @EOB
    tvd_eob = kop + R1 * sin(radians(theta))
    
    # 6. MD @EOB
    if unit == 'ingles':
        md_eob = kop + (theta / bur) * 100
    else:
        md_eob = kop + (theta / bur) * 30
        
    # 7. DH @EOB
    dh_eob = R1 - abs(R1 * cos(radians(theta)))
    
    # 8.1. BC
    tan_len = sqrt(of**2 - fg**2)
    
    # 8.2 MD @SOD
    if unit == 'ingles':
        md_sod = kop + (theta / bur) * 100 + tan_len
    else:
         md_sod = kop + (theta / bur) * 30 + tan_len
            
    # 9. TVD @SOD        
    tvd_sod = tvd_eob + tan_len * abs(cos(radians(theta)))
    
    # 10. DH @SOD
    dh_sod = dh_eob + abs(tan_len * sin(radians(theta)))
    
    # 11. MD Total
    if unit == 'ingles':
        md_total = kop + (theta / bur) * 100 + tan_len + (theta / dor) * 100
    else:
        md_total = kop + (theta / bur) * 30 + tan_len + (theta / dor) * 30
        

    output_S = Output_S(R1=R1, R2=R2, Theta=theta, TVD_EOB=tvd_eob, Md_EOB=md_eob, Dh_EOB=dh_eob, \
                    Tan_len=tan_len, Md_SOD=md_sod, TVD_SOD=tvd_sod, Dh_SOD=dh_sod, Md_total=md_total)

    names = ['R1', 'R2', 'theta', 'tvd_EOB', 'Md_EOB', 'Dh_EOB', 'Lengh_tan', 'Md_SOD', 'tvd_SOD', 'Dh_SOD', 'Md_Total']
    for param , value in zip(names, output_S):
        if param == 'theta':
            print(f"{param} -> {value:.3f} degrees")
        else:
            print(f"{param} -> {value:.3f} ft")

# *Ejercicio 2*

In [19]:
# Data
kop = 6084 #ft
tvd = 12000 #ft
bur = 3 #o/100ft
dor = 2 #o/ft
dh = 3500 #ft

In [20]:
well_S(Data_S(tvd, kop, bur, dor, dh))

R1 -> 1909.860 ft
R2 -> 2864.790 ft
theta -> 39.930 degrees
tvd_EOB -> 7309.854 ft
Md_EOB -> 7415.010 ft
Dh_EOB -> 445.330 ft
Lengh_tan -> 3718.401 ft
Md_SOD -> 11133.411 ft
tvd_SOD -> 10161.220 ft
Dh_SOD -> 2832.005 ft
Md_Total -> 13129.926 ft


## *Horizontal Well Profiles*

<img src="resources/Horizontal_prof.jpg" width=500 height=400 />

In [28]:
# Function to calculate parameters of a Horizontal Well
Data_H = namedtuple("Input", "TVD KOP BUR1 BUR2 DH")
Output_H = namedtuple("Output", "R1 R2 Theta TVD_EOB1 Md_EOB1 Dh_EOB1 Tan_len Md_SOB2 Md_total")

def well_H(data_h:Data_H, unit="ingles"):
    # 1. Definir variables
    tvd = data_h.TVD
    kop = data_h.KOP
    bur1 = data_h.BUR1
    bur2 = data_h.BUR2
    dh = data_h.DH
    
    
    # 3. R1 y R2
    if unit =="ingles":
        R1 = 5729.58 / bur1
        R2 = 5729.58 / bur2
        
    else:
        R1 = 1718.87 / bur1
        R2 = 1718.87 / bur2
        
    # 4. Angulo theta
    # 4.1. Segmento EG
    eg = (tvd - kop) - R2
    
    # 4.2. Segmento EO
    eo = dh - R1
    
    # 4.3. Angulo GOE
    goe = degrees(atan(eg / eo))
    
    # 4.4. Segmento og
    og = sqrt(eg**2 + eo**2)
    
    # 4.5. Segmeto OF
    of = R1 - R2
    
    # 4.6. Angulo GOF
    gof = degrees(acos(of / og))
    
    # 4.7 Angulo theta
    theta = 180 - goe - gof
    
    # 5. TVD @EOB1
    tvd_eob1 = kop + (R1 * sin(radians(theta)))
                      
    # 6. MD @EOB1
    if unit =="ingles":
        md_eob1 = kop + (theta / bur1) * 100
    else:
        md_eob1 = kop + (theta / bur1) * 30
                      
    # 7. DH @EOB1
    dh_eob1 = R1 - (R1 * cos(radians(theta)))
    
    # 8.1 Segmento BC
    tan_len = sqrt(og**2 - of**2)
    
    if unit == "ingles":
        md_sob2 = kop + (theta / bur1) * 100 + tan_len
    else:
        md_sob2 = kop + (theta / bur1) * 30 + tan_len
        
    # 9. MD Total
    if unit == "ingles":
        md_total = kop + (theta / bur1) * 100 + tan_len + ((90 - theta) / bur2) * 100
    else:
        md_total = kop + (theta / bur1) * 30 + tan_len + ((90 - theta) / bur2) * 30
        
        
    output_H = Output_H(R1=R1, R2=R2, Theta=theta, TVD_EOB1=tvd_eob1, Md_EOB1=md_eob1, Dh_EOB1=dh_eob1, Tan_len=tan_len, Md_SOB2=md_sob2, Md_total=md_total)

    names = ['R1', 'R2', 'theta', 'tvd_EOB1', 'Md_EOB1', 'Dh_EOB1', 'Lengh_tan', 'Md_SOB2', 'Md_Total']
    for param , value in zip(names, output_H):
        if param == 'theta':
            print(f"{param} -> {value:.3f} degrees")
        else:
            print(f"{param} -> {value:.3f} ft")
            
            namedtuple("Output", "R1 R2 Theta TVD_EOB1 Md_EOB1 Dh_EOB1 Tan_len Md_SOB2 Md_total")

# *Ejercicio 3*

In [26]:
# Data
tvd = 3800 #ft
kop = 2000 #ft
bur1 = 5.73 #o/100ft
bur2 = 9.55 #o/100ft
dh = 1800 #ft

In [29]:
well_H(Data_H(tvd, kop, bur1, bur2, dh))

R1 -> 999.927 ft
R2 -> 599.956 ft
theta -> 49.792 degrees
tvd_EOB1 -> 2763.645 ft
Md_EOB1 -> 2868.962 ft
Dh_EOB1 -> 354.404 ft
Lengh_tan -> 1385.730 ft
Md_SOB2 -> 4254.692 ft
Md_Total -> 4675.723 ft
